In [180]:
#READ DATA
using CSV, DataFrames, JuMP, Gurobi

df_nut = CSV.read("data_nutrients.csv", DataFrame, delim = ";")
df_rec = CSV.read("data_recipes.csv", DataFrame, delim = ";")
df_recConst = CSV.read("recipes_const.csv", DataFrame, delim = ";")
df_price = CSV.read("price_ing.csv", DataFrame, delim = ";")

,ingredient,price
,String15,Int64
1,chicken,4
2,potatoes,3
3,beef,4
4,rice,2


In [181]:
#DEFINE SETS AND INDICES

#----------------------------------------------------------------------
#set of days (d)
d = 30
D = [j for j in range(1,d)]

#set of meals (m)
m = 3
M = [j for j in range(1,m)]

#set of nutrients (n)
N = unique(df_nut, "nutrients").nutrients

#set of recipes (r)
R = df_rec[:,1]

#set of people (p)
p = 2
P = [j for j in range(1,p)]

#set of ingredients (i)
I = unique(df_recConst, "need").need

#----------------------------------------------------------------------
#indice rmd
rmd = [(l,j,k) for l in R for j in M for k in D]

#indice rmdp
rmdp = [(l,j,k,f) for l in R for j in M for k in D for f in P]

#indice np
np = [(u,f) for u in N for f in P]

#indice nr
nr =[(u,l) for u in N for l in R]

#indice ir
ir = [(row.need,row.recipe) for row in eachrow(df_recConst)]

4-element Vector{Tuple{String15, String31}}:
 ("chicken", "chicken_with_potatoes")
 ("potatoes", "chicken_with_potatoes")
 ("beef", "beef_with_rice")
 ("rice", "beef_with_rice")

In [182]:
#DEFINE DATA

#daily needs of nutrient n by person p
need = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :lowerbound][1] for (j,k) in np)

#daily max of nutrient n by person p
max = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :upperbound][1] for (j,k) in np)

#price of ingredient i per unit of measurement of the ingredient
price = Dict(j => df_price[(df_price.ingredient .== j), :price][1] for j in I)

#quantity of nutrient n granted by one portion of recipe r
nutrec = Dict((j,k) => df_rec[(df_rec.recipe .== k), j][1] for (j,k) in nr)

#quantity of ingredient i needed by one portion of recipe r
recing = Dict((j,k) => df_recConst[(df_recConst.recipe .== k) .& (df_recConst.need .== j), :quantity][1] for (j,k) in ir)

Dict{Tuple{String15, String31}, Int64} with 4 entries:
  ("beef", "beef_with_rice")            => 8
  ("chicken", "chicken_with_potatoes")  => 3
  ("potatoes", "chicken_with_potatoes") => 6
  ("rice", "beef_with_rice")            => 12

In [183]:
#DEFINE AND SHOW MODEL

#model
mdl = Model(with_optimizer(Gurobi.Optimizer))

#decision variables
@variables mdl begin
    X[rmd], Bin
    Q[rmdp]>=0
    Y[I]>=0 #need to understand if it is possible to define some of these as integer and others as float
end

#objective function
@objective(mdl, Min, sum(Y[j]*price[j] for j in I))

#constraints

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-27


4 Y[chicken] + 3 Y[potatoes] + 4 Y[beef] + 2 Y[rice]